In [1]:
import xml.etree.ElementTree as ET

def xmlToTxt(xml_file, txt_file, img_width=504, img_height=747):
    """
    extract bbox information from xml_file into txt_file
    """
    with open(txt_file, 'w+') as f:
        root = ET.parse(xml_file).getroot()
        for object in root.findall('./object'):
            if object[0].text != 'veins_detect':
                continue
            bndbox = object[1]
            xmin = int(bndbox[0].text)
            ymin = int(bndbox[1].text)
            xmax = int(bndbox[2].text)
            ymax = int(bndbox[3].text)

            # print(f'{xmin} {ymin} {xmax} {ymax}')
            
            xcenter = ((xmin + xmax) / 2.0) / img_width
            ycenter = ((ymin + ymax) / 2.0) / img_height
            bbox_width = (xmax - xmin) / img_width
            bbox_height = (ymax - ymin) / img_height

            f.write(f'0 {xcenter} {ycenter} {bbox_width} {bbox_height}\n') # 0 -> class index in class.names
            # print(f'0 {xcenter} {ycenter} {bbox_width} {bbox_height}')

# xml_file = './annotations/59.xml'
# txt_file = '../YOLOv3/data/custom/labels/invivo_91/59.txt'
# xmlToTxt(xml_file, txt_file)

In [2]:
import os
import shutil

def dataTransfer(dataset):
    """
    Split and transfer the raw labelled data into both unet and yolo project directories.
    It is required that the function is implemented under DATA directory because we built relative paths.
    """
    # source dir
    imgs = os.path.join(dataset, 'imgs')
    masks = os.path.join(dataset, 'masks')
    annotations = os.path.join(dataset, 'annotations')
    # dest dir
    imgs_dest_unet = os.path.join('../UNet/data/imgs', dataset); os.makedirs(imgs_dest_unet, exist_ok=True)
    masks_dest = os.path.join('../UNet/data/masks', dataset); os.makedirs(masks_dest, exist_ok=True)
    imgs_dest_yolo = os.path.join('../YOLOv3/data/custom/images/', dataset); os.makedirs(imgs_dest_yolo, exist_ok=True)
    annotations_dest = os.path.join('../YOLOv3/data/custom/labels/', dataset); os.makedirs(annotations_dest, exist_ok=True)

    files = os.listdir(masks)
    count = 0
    for i, fn in enumerate(files):
        # if i == 2: break
        if not fn.endswith('.jpg'):
            continue
        count += 1
        fn_num = fn[:-4]
        
        # source filepaths
        img_fn = os.path.join(imgs, fn_num + '.jpg')
        mask_fn = os.path.join(masks, fn_num + '.jpg')
        annotation_fn = os.path.join(annotations, fn_num + '.xml')
        print(f'fn: {fn} fn_num: {fn_num} img_fn: {img_fn} mask_fn: {mask_fn} annotation_fn: {annotation_fn}')

        # dest filepaths
        img_dest_unet = os.path.join(imgs_dest_unet, fn_num + '.jpg')
        mask_dest = os.path.join(masks_dest, fn_num + '.jpg')
        img_dest_yolo = os.path.join(imgs_dest_yolo, fn_num + '.jpg')
        annotation_dest = os.path.join(annotations_dest, fn_num + '.txt')
        print(f'img_dest_unet: {img_dest_unet} mask_dest: {mask_dest} img_dest_yolo: {img_dest_yolo} annotation_dest: {annotation_dest}')

        # from source to dest
        if os.path.isfile(img_fn) and not os.path.isfile(img_dest_unet): 
            shutil.copyfile(img_fn, img_dest_unet)
            print(f'copy from {img_fn} to {img_dest_unet}')
        if os.path.isfile(mask_fn) and not os.path.isfile(mask_dest): 
            shutil.copyfile(mask_fn, mask_dest)
            print(f'copy from {mask_fn} to {mask_dest}')
        if os.path.isfile(img_fn) and not os.path.isfile(img_dest_yolo):
            shutil.copyfile(img_fn, img_dest_yolo)
            print(f'copy from {img_fn} to {img_dest_yolo}')
        if os.path.isfile(annotation_fn) and not os.path.isfile(annotation_dest):
            xmlToTxt(annotation_fn, annotation_dest)
            print(f'extract bboxs from {annotation_fn} to {annotation_dest}')
        print('')
    print(f'Total samples: {count}')

In [3]:
# dataTransfer('invivo_91')
# dataTransfer('phantom_20')
dataTransfer('phantom_test')

fn: Phantom.jpg fn_num: Phantom img_fn: phantom_test/imgs/Phantom.jpg mask_fn: phantom_test/masks/Phantom.jpg annotation_fn: phantom_test/annotations/Phantom.xml
img_dest_unet: ../UNet/data/imgs/phantom_test/Phantom.jpg mask_dest: ../UNet/data/masks/phantom_test/Phantom.jpg img_dest_yolo: ../YOLOv3/data/custom/images/phantom_test/Phantom.jpg annotation_dest: ../YOLOv3/data/custom/labels/phantom_test/Phantom.txt
copy from phantom_test/imgs/Phantom.jpg to ../UNet/data/imgs/phantom_test/Phantom.jpg
copy from phantom_test/masks/Phantom.jpg to ../UNet/data/masks/phantom_test/Phantom.jpg
copy from phantom_test/imgs/Phantom.jpg to ../YOLOv3/data/custom/images/phantom_test/Phantom.jpg
extract bboxs from phantom_test/annotations/Phantom.xml to ../YOLOv3/data/custom/labels/phantom_test/Phantom.txt

Total samples: 1


In [4]:
dataTransfer('invivo_test')

fn: 44.jpg fn_num: 44 img_fn: invivo_test/imgs/44.jpg mask_fn: invivo_test/masks/44.jpg annotation_fn: invivo_test/annotations/44.xml
img_dest_unet: ../UNet/data/imgs/invivo_test/44.jpg mask_dest: ../UNet/data/masks/invivo_test/44.jpg img_dest_yolo: ../YOLOv3/data/custom/images/invivo_test/44.jpg annotation_dest: ../YOLOv3/data/custom/labels/invivo_test/44.txt
copy from invivo_test/imgs/44.jpg to ../UNet/data/imgs/invivo_test/44.jpg
copy from invivo_test/masks/44.jpg to ../UNet/data/masks/invivo_test/44.jpg
copy from invivo_test/imgs/44.jpg to ../YOLOv3/data/custom/images/invivo_test/44.jpg
extract bboxs from invivo_test/annotations/44.xml to ../YOLOv3/data/custom/labels/invivo_test/44.txt

fn: 33.jpg fn_num: 33 img_fn: invivo_test/imgs/33.jpg mask_fn: invivo_test/masks/33.jpg annotation_fn: invivo_test/annotations/33.xml
img_dest_unet: ../UNet/data/imgs/invivo_test/33.jpg mask_dest: ../UNet/data/masks/invivo_test/33.jpg img_dest_yolo: ../YOLOv3/data/custom/images/invivo_test/33.jpg an